In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
from helpers import *
import seaborn as sns
import math
from proj1_helpers import *
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [ ]:

DATA_TRAIN_PATH = '../data/train.csv'  
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test) # ????
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [ ]:
index0, y0, x_test0 = partition_data(tX, y, 0)
index1, y1, x_test1 = partition_data(tX, y, 1)
index2, y2, x_test2 = partition_data(tX, y, 2)
index3, y3, x_test3 = partition_data(tX, y, 3)






We standardize the 4 batches of data, and a bias component/ feature to each batch.

In [ ]:

std_test0 = np.hstack((np.ones((x_test0.shape[0],1)),standardize_data(x_test0)))
std_test1 = np.hstack((np.ones((x_test1.shape[0],1)),standardize_data(x_test1)))
std_test2 = np.hstack((np.ones((x_test2.shape[0],1)),standardize_data(x_test2)))
std_test3 = np.hstack((np.ones((x_test3.shape[0],1)),standardize_data(x_test3)))


In [ ]:
# np.array(feature_combination(std_test0,np.sin,[0,1]))|
print(std_test0[:,-1])
print(expand_features(std_test0,3).shape)


In [ ]:
expand_features(std_test0,13)

We try to find any correlations between features of the standardized data, by taking a look at their correlation matrices. 

In [ ]:

corr_mat_0 = compute_corr(std_test0[:,1:-1])
corr_mat_1 = compute_corr(std_test1)[:,1:] 
corr_mat_2 = compute_corr(std_test2)[:,1:] 
corr_mat_3 = compute_corr(std_test3)[:,1:] 
    
corrs = [corr_mat_0,corr_mat_1,corr_mat_2,corr_mat_3]

# for elem in corrs:
#     visualize_corr(elem)

We observe that among the 4 batches of partitioned data, the first batch has a feature which is strictly correlated to the other ! 
Furthermore, for the second, thrid , and fourth batch, we observe that some features here and there have very high correlation coefficients, greater than 0.95 ... 

Here, we try to find the  combination of lambda and polynomial expansion degree that results in the most accurate model with respect to cross validation, using ridge regression.

In [ ]:
def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    poly = np.ones((len(x), 1))
    for deg in range(1, degree+1):
        poly = np.c_[poly, np.power(x, deg)]
    return poly[:,1:]

In [ ]:
# X = expand_features(std_test0,2)
# x_o = build_poly(std_test0,2)
# print(std_test0[0:2])
# print("----------------")
# print("Antoine's Version")
# print(x_o[0:2,0:60])
# print("----------------")
# print("Mine")
# print(X[0:2,])
# forward_selection(y0,std_test0)
# cross_validation(11, y0, expand_features(std_test0,12),model='ridge_reg',lambda_=0.5,logging=True)
a = expand_features(std_test0,12)


In [ ]:
degrees= np.arange(1,20)
for d in degrees: 
        print(d)
        b = expand_features(std_test0,d)
        a = cross_validation(11, y0, b, model='lq',logging=True)
        acc.append(a)
                      
print("Best parameters: lambda={}, polynomial degree ={}. Acuracy:{}".format(ind[np.argmax(acc)][0],ind[np.argmax(acc)][1],np.max(acc)))
